In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import datetime

In [ ]:
np.random.seed(30212023)

In [ ]:
# Calculate days between two dates
def diff_dates(date1, date2):
    return abs(date2-date1).days

# Ceiling function
def ceil(n):
    return int(-1 * n // 1 * -1)

def var_calc(ret, alpha, val):

  # http://www.columbia.edu/~amm26/lecture%20files/VaR.pdf
  # VaR calculation used in historical simulation. I did not used a np.percentiel
  # But a more conservative approach. See slides 26 for the above for detail
  # Credit Prof Allan Malz and IEOR 4745 at Columbia U

  m = len(ret)
  # Sort returns in ascending order
  # unannualized since one may want var of different days
  sorted_ret = np.sort(ret)
  i = ceil((1-alpha) * m)
  var = sorted_ret[i]


  # Calculate CVaR at alpha level
  # Find all returns that are worse than the VaR
  worse_returns = ret[ret < var]
  # Calculate the average of all worse returns
  cvar = np.mean(worse_returns)
  
  return -var*val, -cvar*val

In [ ]:
# Data Fetching and cleaning
date_parser = lambda c: pd.to_datetime(c, format='%Y-%m-%d', errors='coerce')

price = pd.read_csv('https://raw.githubusercontent.com/RazerRaymond/ProblemData/main/50MF_Price.csv', sep=',', parse_dates=['as_of'], date_parser=date_parser)
identifier = pd.read_csv('https://raw.githubusercontent.com/RazerRaymond/ProblemData/main/50MF_Identifiers.csv', sep=',')
identifier.rename(columns={'Unnamed: 0':'index'}, inplace=True) 
price.rename(columns={'as_of':'date'}, inplace=True) 
price['date'] = price['date'].dt.date

In [ ]:
# User defined parameters
ts = date(2008, 1, 1) # start date
te = date(2011, 1, 1) # end date
d = 5 # number of mutual funds in the portfolio
tau = 365 # time horizon, e.g. 365 for a year
delta = 7  # rolling period e.g. 30 for monthly 
alpha = 0.95 # confidence level
m = 500000 # total investment

In [ ]:
# randomly pick which stock to pick
choice_index = np.random.choice(range(50), d, replace=False)

In [ ]:
def varPorfolio(ts, te, choice_index, tau, delta, alpha, m):
  # get names of mutual funds
  mf_col = identifier.loc[choice_index, 'ask_id'].values
  names = ', '.join(identifier.loc[choice_index, 'sec_name'].values)
  # Calculate L
  L = (diff_dates(te,ts) - tau + 1) // delta # use // for floor

  # slice the dataframe
  ts_i = price.loc[price['date'] == ts].index.values[0]
  te_i = price.loc[price['date'] == te].index.values[0]
  df = price.iloc[ts_i:te_i+1].reset_index(drop=True)

  # Calculate returns
  dateColumn = df['date']
  ret_df = df.drop('date', axis=1).pct_change(periods=tau)

  # Shift the returns back by M periods to place them at their original location
  ret_df = ret_df.shift(-tau).dropna()
  ret_df.insert(0, 'date', dateColumn)
  target_index = np.arange(0, (L*delta), delta)

  # Only keep data of interest
  ret_df = ret_df.iloc[target_index]
  ret_df = ret_df.loc[:, mf_col]
  VaR, CVaR = 0, 0
  for col in ret_df.columns:
      series = ret_df[col]
      a, b = var_calc(series, alpha, m/d) 
      VaR += a
      CVaR += b
  print(f'We picked {len(choice_index)} assets: {names}')
  return VaR, CVaR

In [ ]:
VaR, CVaR = varPorfolio(ts, te, choice_index, tau, delta, alpha, m)
print(f'At {alpha*100}% confident level, VaR of the portfolio is {VaR}')
print(f'At {alpha*100}% confident level, CVaR of the portfolio is {CVaR}')

We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
At 95.0% confident level, VaR of the portfolio is 79439.8299223685
At 95.0% confident level, CVaR of the portfolio is 88547.50912865394


In [ ]:
taul = [1, 7, 30, 90, 365]
deltal = [1, 7, 30, 90, 365]

columns = [1, 7, 30, 90, 365] # tau values
rows = [1, 7, 30, 90, 365] # delta values

dfV = pd.DataFrame(index=rows, columns=columns)
dfC = pd.DataFrame(index=rows, columns=columns)

for i in taul:
  for j in deltal:
    if(i >= j):
      a, b = varPorfolio(ts, te, choice_index, i, j, alpha, m)
      dfV.loc[j, i] = a
      dfC.loc[j, i] = b
dfV

We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (SIMT), Fidelity® Municipal Income, Lord Abbett Income A
We picked 5 assets: PGIM Corporate Bond Z, USAA High Income, SEI Mid-Cap F (

,1,7,30,90,365
1,4282.592525,12537.848075,30881.589891,75386.349521,81414.579572
7,NaN,11849.479581,28420.079096,72323.94538,79439.829922
30,NaN,NaN,21862.235295,63151.810865,77505.239613
90,NaN,NaN,NaN,27083.47677,73803.825781
365,NaN,NaN,NaN,NaN,-148457.81527


In [ ]:
dfC

,1,7,30,90,365
1,7602.125921,21518.286413,52471.544236,91004.19283,89432.670667
7,NaN,20280.462712,53238.277034,89365.839066,88547.509129
30,NaN,NaN,48668.630304,88736.39319,88472.102748
90,NaN,NaN,NaN,66891.221689,87652.802467
365,NaN,NaN,NaN,NaN,86611.037437
